In [20]:
!pip install qiskit qiskit-aer

import numpy as np
from qiskit import QuantumCircuit, transpile
import matplotlib.pyplot as plt
from qiskit_aer import AerSimulator
from qiskit_aer.noise import NoiseModel, depolarizing_error, ReadoutError
# from qrisp.interface import IQMBackend # Uncomment for real hardware

def run_cluster_witness_theorem_2(n_qubits, backend, shots=2000):
    """
    Implements the Cluster State Witness from Toth & Guhne Theorem 2.
    Formula: W = 3*I - 2*(Prob_All_Even_Satisfied + Prob_All_Odd_Satisfied)
    Target: W < 0 implies Genuine Multipartite Entanglement.
    """

    # --- 1. PREPARE THE STATE ---
    # We create the "perfect" linear cluster state to test against
    qc_base = QuantumCircuit(n_qubits)
    qc_base.h(range(n_qubits)) # Initialize |+>
    # Apply CZ gates (Linear Topology matches IQM Garnet)
    for i in range(0, n_qubits - 1, 2): qc_base.cz(i, i+1) # Even edges
    for i in range(1, n_qubits - 1, 2): qc_base.cz(i, i+1) # Odd edges

    # --- 2. DEFINE THE TWO SETTINGS (Fig 1c from Paper) ---

    # Setting A: Measure stabilizers centered on EVEN QISKIT INDICES (0, 2, 4...)
    # Corresponds to Paper's "Odd k" (S1, S3...)
    # Pattern: X Z X Z ...
    qc_A = qc_base.copy()
    # To measure X, apply H. To measure Z, do nothing.
    for i in range(0, n_qubits, 2):
        qc_A.h(i)
    qc_A.measure_all()

    # Setting B: Measure stabilizers centered on ODD QISKIT INDICES (1, 3, 5...)
    # Corresponds to Paper's "Even k" (S2, S4...)
    # Pattern: Z X Z X ...
    qc_B = qc_base.copy()
    for i in range(1, n_qubits, 2):
        qc_B.h(i)
    qc_B.measure_all()

    # --- 3. RUN EXPERIMENT ---
    print(f"Running Witness for N={n_qubits}...")
    # fig_A = qc_A.draw(output="mpl")
    # plt.show()
    job_A = backend.run(transpile(qc_A, backend), shots=shots)
    # fig_B = qc_B.draw(output="mpl")
    # plt.show()
    job_B = backend.run(transpile(qc_B, backend), shots=shots)
    counts_A = job_A.result().get_counts()
    counts_B = job_B.result().get_counts()

    # --- 4. CALCULATE PROBABILITIES (The "Product" Terms) ---

    # Term 1: Probability that ALL "Odd k" stabilizers are +1
    # Stabilizers: S_0, S_2, S_4 (Qiskit indices)
    success_count_A = 0
    for bitstring, count in counts_A.items():
        all_satisfied = True
        # Check every stabilizer in this group
        for i in range(0, n_qubits, 2):
            # Calculate Parity of S_i = Z_{i-1} X_i Z_{i+1}
            # Note: Qiskit bitstrings are reversed! bit[0] is qubit N-1.
            parity = 1
            # Center (X_i)
            if bitstring[n_qubits - 1 - i] == '1': parity *= -1
            # Left (Z_{i-1})
            if i > 0 and bitstring[n_qubits - 1 - (i-1)] == '1': parity *= -1
            # Right (Z_{i+1})
            if i < n_qubits - 1 and bitstring[n_qubits - 1 - (i+1)] == '1': parity *= -1

            if parity == -1: # Rule Broken!
                all_satisfied = False
                break

        if all_satisfied:
            success_count_A += count

    prob_A = success_count_A / shots

    # Term 2: Probability that ALL "Even k" stabilizers are +1
    # Stabilizers: S_1, S_3, S_5 (Qiskit indices)
    success_count_B = 0
    for bitstring, count in counts_B.items():
        all_satisfied = True
        for i in range(1, n_qubits, 2):
            parity = 1
            # Center (X_i)
            if bitstring[n_qubits - 1 - i] == '1': parity *= -1
            # Left (Z_{i-1})
            if bitstring[n_qubits - 1 - (i-1)] == '1': parity *= -1
            # Right (Z_{i+1})
            if i < n_qubits - 1 and bitstring[n_qubits - 1 - (i+1)] == '1': parity *= -1

            if parity == -1:
                all_satisfied = False
                break

        if all_satisfied:
            success_count_B += count

    prob_B = success_count_B / shots

    # --- 5. COMPUTE WITNESS VALUE ---
    # W = 3 - 2 * (P_A + P_B)
    w_value = 3 - 2 * (prob_A + prob_B)

    return w_value, prob_A, prob_B

def create_noise_model(single_qubit_error=0.001, two_qubit_error=0.01, readout_error=0.02):
    """
    Creates a realistic noise model with gate and measurement errors.

    Args:
        single_qubit_error: Error probability for single-qubit gates (e.g., H)
        two_qubit_error: Error probability for two-qubit gates (e.g., CZ)
        readout_error: Measurement error probability
    """
    noise_model = NoiseModel()

    # Single-qubit gate errors (depolarizing)
    single_qubit_noise = depolarizing_error(single_qubit_error, 1)
    noise_model.add_all_qubit_quantum_error(single_qubit_noise, ['h'])

    # Two-qubit gate errors (depolarizing)
    two_qubit_noise = depolarizing_error(two_qubit_error, 2)
    noise_model.add_all_qubit_quantum_error(two_qubit_noise, ['cz', 'cx'])

    # Readout errors
    # Probability of measuring 1 when state is 0, and measuring 0 when state is 1
    readout_probs = [[1 - readout_error, readout_error],
                     [readout_error, 1 - readout_error]]
    readout_noise = ReadoutError(readout_probs)
    noise_model.add_all_qubit_readout_error(readout_noise)

    return noise_model

# --- MOCK EXECUTION ---
# Create a noisy simulator
print("=== Noisy Simulation ===")
print("Noise Parameters:")
print("  Single-qubit gates (H): 0.1% error")
print("  Two-qubit gates (CZ): 1.0% error")
print("  Readout: 2.0% error\n")

noise_model = create_noise_model(
     single_qubit_error=0.001,
        two_qubit_error=0.02,
        readout_error=0.002
)
backend_sim = AerSimulator(noise_model=noise_model)

# Test for N=4, 6, 8, 10
for n in [4, 6, 8, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 22, 25]: # Changed from 30 to 25
    w, pA, pB = run_cluster_witness_theorem_2(n, backend_sim)
    print(f"N={n}: Witness = {w:.3f} (P_odd={pA:.2f}, P_even={pB:.2f})")
    # Expected for perfect state: W = 3 - 2(1+1) = -1.0
    # With noise, W will be closer to 0 (less negative)

=== Noisy Simulation ===
Noise Parameters:
  Single-qubit gates (H): 0.1% error
  Two-qubit gates (CZ): 1.0% error
  Readout: 2.0% error

Running Witness for N=4...
N=4: Witness = -0.828 (P_odd=0.96, P_even=0.95)
Running Witness for N=6...
N=6: Witness = -0.700 (P_odd=0.92, P_even=0.93)
Running Witness for N=8...
N=8: Witness = -0.599 (P_odd=0.90, P_even=0.90)
Running Witness for N=10...
N=10: Witness = -0.507 (P_odd=0.88, P_even=0.87)
Running Witness for N=11...
N=11: Witness = -0.462 (P_odd=0.87, P_even=0.86)
Running Witness for N=12...
N=12: Witness = -0.360 (P_odd=0.84, P_even=0.84)
Running Witness for N=13...
N=13: Witness = -0.344 (P_odd=0.83, P_even=0.84)
Running Witness for N=14...
N=14: Witness = -0.287 (P_odd=0.82, P_even=0.82)
Running Witness for N=15...
N=15: Witness = -0.245 (P_odd=0.81, P_even=0.81)
Running Witness for N=16...
N=16: Witness = -0.212 (P_odd=0.81, P_even=0.80)
Running Witness for N=17...
N=17: Witness = -0.125 (P_odd=0.79, P_even=0.78)
Running Witness for N

In [22]:
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
from qiskit_aer.noise import NoiseModel, depolarizing_error, ReadoutError

# ============================================================
# Helper functions (bit order + stabilizers)
# ============================================================

def bit_at_qubit(bitstring, q, n):
    # Qiskit: leftmost bit is highest index
    return 1 if bitstring[n - 1 - q] == "1" else 0

def stabilizer_Ki(bitstring, i, n):
    # K_i = Z_{i-1} X_i Z_{i+1}
    parity = bit_at_qubit(bitstring, i, n)
    if i > 0:
        parity ^= bit_at_qubit(bitstring, i-1, n)
    if i < n-1:
        parity ^= bit_at_qubit(bitstring, i+1, n)
    return +1 if parity == 0 else -1

def apply_stabilizer_postselection(counts, n, check_indices):
    kept = {}
    total = sum(counts.values())
    rejected = 0

    for b, c in counts.items():
        if all(stabilizer_Ki(b, i, n) == +1 for i in check_indices):
            kept[b] = kept.get(b, 0) + c
        else:
            rejected += c

    return kept, rejected / total if total else 0.0

# ============================================================
# Cluster witness experiment (with stabilizer postselection)
# ============================================================

def run_cluster_witness(
    n,
    backend,
    shots=2000,
    checks_A=None,
    checks_B=None
):
    checks_A = checks_A or []
    checks_B = checks_B or []

    # --- Prepare linear cluster state ---
    qc_base = QuantumCircuit(n)
    qc_base.h(range(n))
    for i in range(n-1):
        qc_base.cz(i, i+1)

    # --- Setting A: measure X on even sites ---
    qc_A = qc_base.copy()
    for i in range(0, n, 2):
        qc_A.h(i)
    qc_A.measure_all()

    # --- Setting B: measure X on odd sites ---
    qc_B = qc_base.copy()
    for i in range(1, n, 2):
        qc_B.h(i)
    qc_B.measure_all()

    # --- Run circuits ---
    jobA = backend.run(transpile(qc_A, backend), shots=shots)
    jobB = backend.run(transpile(qc_B, backend), shots=shots)

    counts_A = jobA.result().get_counts()
    counts_B = jobB.result().get_counts()

    # --- Postselection (ONLY stabilizers!) ---
    counts_A, rejA = apply_stabilizer_postselection(counts_A, n, checks_A)
    counts_B, rejB = apply_stabilizer_postselection(counts_B, n, checks_B)

    # --- Compute Theorem-2 probabilities ---
    def prob_all(indices, counts):
        total = sum(counts.values())
        if total == 0:
            return 0.0
        good = 0
        for b, c in counts.items():
            if all(stabilizer_Ki(b, i, n) == +1 for i in indices):
                good += c
        return good / total

    even_indices = [i for i in range(0, n, 2) if i not in checks_A]
    odd_indices  = [i for i in range(1, n, 2) if i not in checks_B]

    P_A = prob_all(even_indices, counts_A)
    P_B = prob_all(odd_indices, counts_B)

    W = 3 - 2 * (P_A + P_B)

    return W, P_A, P_B, rejA, rejB

# ============================================================
# Noise model (same spirit as your original)
# ============================================================

def make_noise():
    noise = NoiseModel()
    noise.add_all_qubit_quantum_error(depolarizing_error(0.001, 1), ['h'])
    noise.add_all_qubit_quantum_error(depolarizing_error(0.02, 2), ['cz'])
    noise.add_all_qubit_readout_error(
        ReadoutError([[0.998, 0.002], [0.002, 0.998]])
    )
    return noise

backend = AerSimulator(noise_model=make_noise())

# ============================================================
# Run comparison
# ============================================================

print("\n=== Cluster witness with stabilizer postselection ===")

for n in [4, 6, 8, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 22, 25]:
    W0, *_ = run_cluster_witness(n, backend, checks_A=[], checks_B=[])
    Wp, _, _, rA, rB = run_cluster_witness(
        n,
        backend,
        checks_A=[2] if n > 3 else [],
        checks_B=[1] if n > 2 else []
    )

    print(
        f"N={n:2d} | "
        f"W(no ps)={W0:+.3f} | "
        f"W(ps)={Wp:+.3f} | "
        f"rejA={rA:.2f}, rejB={rB:.2f}"
    )



=== Cluster witness with stabilizer postselection ===
N= 4 | W(no ps)=-0.815 | W(ps)=-0.946 | rejA=0.03, rejB=0.03
N= 6 | W(no ps)=-0.701 | W(ps)=-0.862 | rejA=0.04, rejB=0.03
N= 8 | W(no ps)=-0.621 | W(ps)=-0.753 | rejA=0.04, rejB=0.03
N=10 | W(no ps)=-0.546 | W(ps)=-0.649 | rejA=0.03, rejB=0.03
N=11 | W(no ps)=-0.444 | W(ps)=-0.568 | rejA=0.03, rejB=0.04
N=12 | W(no ps)=-0.407 | W(ps)=-0.515 | rejA=0.03, rejB=0.03
N=13 | W(no ps)=-0.328 | W(ps)=-0.465 | rejA=0.03, rejB=0.03
N=14 | W(no ps)=-0.264 | W(ps)=-0.434 | rejA=0.03, rejB=0.04
N=15 | W(no ps)=-0.250 | W(ps)=-0.338 | rejA=0.03, rejB=0.04
N=16 | W(no ps)=-0.229 | W(ps)=-0.325 | rejA=0.04, rejB=0.04
N=17 | W(no ps)=-0.150 | W(ps)=-0.260 | rejA=0.04, rejB=0.03
N=18 | W(no ps)=-0.110 | W(ps)=-0.216 | rejA=0.03, rejB=0.03
N=19 | W(no ps)=-0.082 | W(ps)=-0.188 | rejA=0.03, rejB=0.03
N=20 | W(no ps)=+0.011 | W(ps)=-0.107 | rejA=0.04, rejB=0.04
N=22 | W(no ps)=+0.089 | W(ps)=-0.042 | rejA=0.04, rejB=0.03
N=25 | W(no ps)=+0.201 | W(ps)